In [ ]:
import pandas as pd
import numpy as np

pd.set_option('display.max_rows',None)
pd.set_option('display.max_colwidth', None)

In [ ]:
df1945 = pd.read_csv('/content/1945_Standardized_Final - 1945_Standardized.csv.csv')
df1975 = pd.read_csv('/content/1975_Standardized_Final - 1975_Standardized.csv.csv')
df1985 = pd.read_csv('/content/1985_Standardized_Final - 1985_s.csv.csv')

<ipython-input-4-f818104b6b15>:2: DtypeWarning: Columns (12,13,14,15) have mixed types. Specify dtype option on import or set low_memory=False.
  df1975 = pd.read_csv('/content/1975_Standardized_Final - 1975_Standardized.csv.csv')


In [ ]:
df1945_ = df1945[['ID_1945','Standardized_Address_1945']]
df1975_ = df1975[['ID_1975','Standardized_Address_1975']]
df1985_ = df1985[['ID_1985','Standardized_Address_1985']]

In [ ]:
df1945_.shape

(5610, 2)

In [ ]:
df1945_[df1945_['Standardized_Address_1945'].notna()].shape

(2705, 2)

In [ ]:
df1975_.shape

(61600, 2)

In [ ]:
df1975_[df1975_['Standardized_Address_1975'].notna()].shape

(40288, 2)

In [ ]:
df1985_.shape

(25977, 2)

In [ ]:
df1985_[df1985_['Standardized_Address_1985'].notna()].shape

(18603, 2)

In [ ]:
# Finding out the common standardized addresses(Standardized_Address_1945, Standardized_Address_1975, Standardized_Address_1985) in df1945_, df1975_, df1985_ Standardized_Address_1985

# Take Standardized_Address_1945 column from df1945_ and check if a row from that column is matching with any rows in Standardized_Address_1975 and/or Standardized_Address_1985.
# If matching, then create a new dataframe in the first column take the ID_1945. In the second column, take the Standardized_Address_1945, in the third column take 1975_matches and as the forth column take 1985_matches.
# Save the values of ID_1975 in 1975_matches for the matched Standardized_Address_1945
# Save the values of ID_1985 in 1985_matches for the matched Standardized_Address_1945
# Deal with NaNs in all the dfs.

In [ ]:
def find_common_addresses(dfs, address_columns, id_columns):
    """
    Find common addresses across multiple dataframes.

    :param dfs: List of dataframes to compare
    :param address_columns: List of column names containing the addresses in each dataframe
    :param id_columns: List of column names containing the IDs in each dataframe
    :return: A dataframe with common addresses and their corresponding IDs from each dataframe
    """
    # Check if the length of dfs, address_columns, and id_columns are equal
    if not (len(dfs) == len(address_columns) == len(id_columns)):
        raise ValueError("The length of dfs, address_columns, and id_columns must be the same")

    # Group each dataframe by its address column and aggregate the IDs into lists
    grouped_dataframes = [
        df.groupby(address)[id].apply(list).reset_index()
        for df, address, id in zip(dfs, address_columns, id_columns)
    ]

    # Initialize the result with the first grouped dataframe
    result_df = grouped_dataframes[0]
    result_df.columns = ['Common_Address', 'IDs_1']

    # Iteratively merge the grouped dataframes to find common addresses
    for i, grouped_df in enumerate(grouped_dataframes[1:], start=2):
        grouped_df.columns = ['Common_Address', f'IDs_{i}']
        result_df = pd.merge(result_df, grouped_df, on='Common_Address', how='inner')

    return result_df

In [ ]:
result_df_all = find_common_addresses(
    [df1945_, df1975_, df1985_],
    ['Standardized_Address_1945', 'Standardized_Address_1975', 'Standardized_Address_1985'],
    ['ID_1945', 'ID_1975', 'ID_1985']
)

result_df_all.shape

(304, 4)

In [ ]:
id_counts = {col: sum(len(ids) for ids in result_df_all[col]) for col in result_df_all.columns if col.startswith('IDs_')}

id_counts

{'IDs_1': 363, 'IDs_2': 680, 'IDs_3': 469}

In [ ]:
result_df_1945_1975 = find_common_addresses(
    [df1945_, df1975_],
    ['Standardized_Address_1945', 'Standardized_Address_1975'],
    ['ID_1945', 'ID_1975']
)

result_df_1945_1975.shape

(877, 3)

In [ ]:
id_counts = {col: sum(len(ids) for ids in result_df_1945_1975[col]) for col in result_df_1945_1975.columns if col.startswith('IDs_')}

id_counts

{'IDs_1': 1037, 'IDs_2': 1496}

In [ ]:
result_df_1945_1985 = find_common_addresses(
    [df1945_, df1985_],
    ['Standardized_Address_1945', 'Standardized_Address_1985'],
    ['ID_1945', 'ID_1985']
)

result_df_1945_1985.shape

(492, 3)

In [ ]:
id_counts = {col: sum(len(ids) for ids in result_df_1945_1985[col]) for col in result_df_1945_1985.columns if col.startswith('IDs_')}

id_counts

{'IDs_1': 590, 'IDs_2': 740}

In [ ]:
# Usage example
result_df_1975_1985 = find_common_addresses(
    [df1975_, df1985_],
    ['Standardized_Address_1975', 'Standardized_Address_1985'],
    ['ID_1975', 'ID_1985']
)

result_df_1975_1985.shape

(6990, 3)

In [ ]:
id_counts = {col: sum(len(ids) for ids in result_df_1975_1985[col]) for col in result_df_1975_1985.columns if col.startswith('IDs_')}

id_counts

{'IDs_1': 13071, 'IDs_2': 10301}

In [ ]:
result_df_all.to_csv('common_in_all.csv', index=False)